This tutorial follows the samples and explanations of https://www.pinecone.io/learn/series/langchain/langchain-intro/

# Chapter 1. Introduction to LangChain

In [1]:
import os

from langchain import PromptTemplate, LLMChain
from langchain.llms import AzureOpenAI

In [2]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [3]:
# PromptTemplate builds the general structure for a traditional prompt
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template = template,
                        input_variables = ['question'])

question = "Tell me about the band Ghost."

In [4]:
# Specify the model
llm = AzureOpenAI(deployment_name = "text-davinci-003",
                  model_name = "text-davinci-003")

# Run the prompt with the LLM
llm_chain = LLMChain(prompt = prompt,
                     llm = llm)

In [5]:
print(llm_chain.run(question))

 Ghost is a Swedish heavy metal band that was formed in Linköping, Sweden in 2008. They are known for their unique blend of heavy metal, hard rock, and doom metal with occult and horror-inspired lyrics. The band's musical style has been described as a mix of classic '70s and '80s hard rock and heavy metal with a modern touch. The band's lineup consists of vocalist Tobias Forge, the band's leader, and four additional musicians who use the stage names of Nameless Ghouls. Ghost has released three studio albums, two live albums, and a number of singles and EPs. Their most recent album, Prequelle, was released in 2018.


In [6]:
# Run multiple questions
qs = [{'question': "If I'm 6ft 4 inches, how tall am I in centimeters?"},
      {'question': 'What is LangChain?'},
      {'question': 'How many books does Tolkien wrote related to the Lord of the Rings?'},
      {'question': 'How many pokemons exist?'}]

llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' 6ft 4 inches is equal to 193.04 centimeters.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' LangChain is a decentralized language learning platform that uses blockchain technology to enable users to learn languages from native speakers and earn rewards for their language learning efforts. The platform connects language learners with native speakers who can provide personalized language lessons. Learners can earn rewards in the form of cryptocurrency for completing lessons and helping others learn languages.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Tolkien wrote three books related to The Lord of the Rings: The Fellowship of the Ring, The Two Towers, and The Return of the King.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' As of July 2020, there are 898 known species of Pokémon.', generation_info={'finish_reason': 'stop', 'logpro

In [7]:
# Get just the answers
qs_str = ["If I'm 6ft 4 inches, how tall am I in centimeters?\n" +
          'What is LangChain?\n' +
          'How many books does Tolkien wrote related to the Lord of the Rings?\n' +
          'How many pokemons exist?']

print(llm_chain.run(qs_str))


6ft 4 inches is approximately 193 centimeters.
LangChain is an online language learning platform that uses AI and NLP technology to help users learn a language in an interactive and fun way.

J.R.R. Tolkien wrote three books related to the Lord of the Rings: The Fellowship of the Ring, The Two Towers, and The Return of the King.

There are currently 809 different species of Pokemon.


# Chapter 2. Prompt Templates and the Art of Prompts

In [8]:
import os

from langchain import PromptTemplate, FewShotPromptTemplate, LLMChain
from langchain.llms import AzureOpenAI
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [9]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [10]:
prompt = """Answer the question based on the context below.
If the question cannot be answered using the information provided answer with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly useful for developers
building NLP enabled applications. These models can be accessed via Hugging Face's `transformers`
library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library.

Question: which libraries and omdel providers offer LLMs?

Answer: """

You can run the prompt directly after defining the LLM model you want to implement.

In [11]:
# Specify the model
llm = AzureOpenAI(deployment_name = "text-davinci-003",
                  model_name = "text-davinci-003")

# Run the prompt with the LLM
print(llm(prompt))

 Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


In case you have prompts with **dynamic inputs**, the best approach is to use **prompt templates**.

In [12]:
template = """Answer the question based on the context below.
If the question cannot be answered using the information provided answer with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly useful for developers
building NLP enabled applications. These models can be accessed via Hugging Face's `transformers`
library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

In [13]:
prompt_template = PromptTemplate(input_variables = ["query"],
                                 template = template)

In [14]:
# Modify the prompt template based on the dynamic input (the user question)
print(prompt_template.format(query = "Which libraries and model providers offer LLMs?"))

Answer the question based on the context below.
If the question cannot be answered using the information provided answer with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly useful for developers
building NLP enabled applications. These models can be accessed via Hugging Face's `transformers`
library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [15]:
# Run directly the prompt
print(llm(prompt_template.format(
            query = "Which libraries and model providers offer LLMs?")))

 Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


## Few Shot Prompt Templates

In [16]:
# Use of few-shot samples for improving the prompt
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:

User: What is the meaning of life?
AI: """

llm.temperature = 1.0
print(llm(prompt))

 To be the best version of yourself that you can be!


In [17]:
# Giving more examples of input-output improves the quality of response
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

llm.temperature = 1.0
print(llm(prompt))

 The meaning of life is to find your own purpose and live it to the fullest!


In [18]:
# Formalize this process with a specific template
examples = [{"query": 'How are you?',
             "answer": "I can't complain but sometimes I still do."},
            {"query": "What time is it?",
             "answer": "It's time to get a watch."}]

example_template = """User: {query}
AI: {answer}"""

example_prompt = PromptTemplate(input_variables = ['query', 'answer'],
                                template = example_template)

prefix = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:"""

suffix = """User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(examples = examples,
                                                 example_prompt = example_prompt,
                                                 prefix = prefix,
                                                 suffix = suffix,
                                                 input_variables = ['query'],
                                                 example_separator = '\n\n')

In [19]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: 


In [20]:
# Limit the number of examples for the prompt
examples = [{"query": "How are you?",
             "answer": "I can't complain but sometimes I still do."},
            {"query": "What time is it?",
             "answer": "It's time to get a watch."},
            {"query": "What is the meaning of life?",
             "answer": "42!"},
            {"query": "What is the weather like today?",
             "answer": "Cloudy with a chance of memes."},
            {"query": "What is your favorite movie?",
             "answer": "Terminator"},
            {"query": "Who is your best friend?",
             "answer": "Siri. We have spirited debates about the meaning of life."},
            {"query": "What should I do today?",
             "answer": "Stop talking to chatbots on the internet and go outside."}]

In [21]:
example_selector = LengthBasedExampleSelector(examples = examples,
                                              example_prompt = example_prompt,
                                              max_length = 50) #Maximum number of words, not tokens

In [22]:
dynamic_prompt_template = FewShotPromptTemplate(example_selector = example_selector,
                                                example_prompt = example_prompt,
                                                prefix = prefix,
                                                suffix = suffix,
                                                input_variables = ['query'],
                                                example_separator = '\n')

In [23]:
query = "How do birds fly?"

print(dynamic_prompt_template.format(query=query))

The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:
User: How are you?
AI: I can't complain but sometimes I still do.
User: What time is it?
AI: It's time to get a watch.
User: What is the meaning of life?
AI: 42!
User: How do birds fly?
AI: 


In [24]:
# longer questions limit the given example questions
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions.
Here are some examples:
User: How are you?
AI: I can't complain but sometimes I still do.
User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


# Chapter 3. Building Composable Pipelines with Chains

A chain is basically a pipeline that processes an input by using a specific combination of primitives. Intuitively, it can be thought of as a 'step' that performs a certain set of operations on an input and returns the result. They can be anything from a prompt-based pass through a LLM to applying a Python function to an text.

In [25]:
import os
import re
import inspect

from langchain import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [26]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [27]:
# Specify the model
llm = AzureOpenAI(deployment_name = "text-davinci-003",
                  model_name = "text-davinci-003",
                  temperature = 0)

In [28]:
# Function for retrieving used tokens
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens")
        
    return result

## Utility Chains

Chains that are usually used to extract a specific answer from a LLM with a very narrow purpose and are ready to be used out of the box.

In [29]:
# Prepare the LLM to do math
llm_math = LLMMathChain(llm = llm,
                        verbose = True) #Show the different steps that are run on the chain
count_tokens(llm_math,
             "What is 13 raised to the 0.3432 power?")



> Entering new LLMMathChain chain...

D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(



What is 13 raised to the 0.3432 power?
```text
13**0.3432
```
...numexpr.evaluate("13**0.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 267 tokens


'Answer: 2.4116004626599237'

In [30]:
# Check the chain prompt executed above
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [31]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



## Generic Chains

Chains that are used as building blocks for other chains but cannot be used out of the box on their own.

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [32]:
def transform_func(inputs: dict) -> dict:
    text = inputs['text']
    
    #Replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    
    return {'output_text': text}

In [33]:
clean_extra_spaces_chain = TransformChain(input_variables = ['text'],
                                          output_variables = ['output_text'],
                                          transform = transform_func)

In [34]:
clean_extra_spaces_chain.run("A random text  with   some irregular spacing.\n\n\n     Another one   here as well.")

'A random text with some irregular spacing.\n Another one here as well.'

In [35]:
# Use the previous TransformChain to paraphrase input texts with LLMs
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """

prompt = PromptTemplate(input_variables = ['style', 'output_text'],
                        template = template)

In [36]:
style_paraphrase_chain = LLMChain(llm = llm,
                                  prompt = prompt,
                                  output_key = 'final_output')

In [37]:
sequential_chain = SequentialChain(chains = [clean_extra_spaces_chain,
                                             style_paraphrase_chain],
                                   input_variables = ['text', 'style'],
                                   output_variables = ['final_output'])

In [38]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

In [39]:
count_tokens(sequential_chain,
             {'text': input_text,
              'style': 'a 90s rapper'})

Spent a total of 163 tokens


"\nChains let us link up multiple pieces to make one dope app. Like, we can take user input, style it up with a PromptTemplate, then pass it to an LLM. We can get even more creative by combining multiple chains or mixin' chains with other components."

# Chapter 4. Conversational Memory

In [40]:
import os

from langchain import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationSummaryBufferMemory)
from langchain.callbacks import get_openai_callback

In [41]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [42]:
# Specify the model
llm = AzureOpenAI(deployment_name = "text-davinci-003",
                  model_name = "text-davinci-003",
                  temperature = 0)

# Initialize the conversation chain
conversation = ConversationChain(llm = llm)

In [43]:
# Check prompt template of conversation template
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## Conversation Buffer Memory

Is the most straightforward conversational memory in LangChain. The raw input of the past conversation between the human and AI is passed — in its raw form — to the {history} parameter.

In [44]:
conversation_buf = ConversationChain(llm = llm,
                                     memory = ConversationBufferMemory())

In [45]:
conversation_buf("Hello AI!")

{'input': 'Hello AI!',
 'history': '',
 'response': " Hi there! It's nice to meet you. How can I help you today?"}

In [46]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [47]:
count_tokens(conversation_buf,
             "My interest here is to explore the potential of integrating Large Language Models with external knowledge")

Spent a total of 143 tokens


" Interesting! That sounds like a fascinating topic. I'm not sure I know much about it, but I'm sure I can help you find some resources. What kind of resources are you looking for?"

In [48]:
count_tokens(conversation_buf,
             "I just want to analyze the different possibilities. What can you think of?")

Spent a total of 247 tokens


" Hmm, that's a great question. I'm not sure I have a definitive answer, but I can think of a few possibilities. For example, you could explore the potential of using large language models to generate more accurate natural language processing results, or you could look into using them to create more accurate machine translation systems. You could also explore the potential of using them to create more accurate text summarization systems."

In [49]:
count_tokens(conversation_buf, 
             "Which data source types could be used to give context to the model?")

Spent a total of 364 tokens


" That's a great question. There are a few different types of data sources that could be used to give context to a large language model. For example, you could use text corpora, which are collections of text documents that can be used to train the model. You could also use structured data sources, such as databases or spreadsheets, to provide additional context. Additionally, you could use unstructured data sources, such as audio or video recordings, to provide additional context."

In [50]:
# Check the buffer memory
print(conversation_buf.memory.buffer)

Human: Hello AI!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! That sounds like a fascinating topic. I'm not sure I know much about it, but I'm sure I can help you find some resources. What kind of resources are you looking for?
Human: I just want to analyze the different possibilities. What can you think of?
AI:  Hmm, that's a great question. I'm not sure I have a definitive answer, but I can think of a few possibilities. For example, you could explore the potential of using large language models to generate more accurate natural language processing results, or you could look into using them to create more accurate machine translation systems. You could also explore the potential of using them to create more accurate text summarization systems.
Human: Which data source types could be used to give context to the model?
AI:  That's a great 

## Conversation Summary Memory

This form of memory summarizes the conversation history before it is passed to the {history} parameter.

In [51]:
conversation = ConversationChain(llm=llm,
                                 memory=ConversationSummaryMemory(llm=llm))

In [52]:
print(conversation.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [53]:
# Can use conversation("Good morning AI!") without checking on total tokens
count_tokens(conversation, 
             "Good morning AI!")

Spent a total of 283 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [54]:
count_tokens(conversation,
             "My interest here is to explore the potential of integrating Large Language Models with external knowledge")

Spent a total of 420 tokens


" That sounds like an interesting project! I'm familiar with Large Language Models, but I'm not sure how they could be integrated with external knowledge. Could you tell me more about what you have in mind?"

In [55]:
count_tokens(conversation,
             "I just want to analyze the different possibilities. What can you think of?")

Spent a total of 629 tokens


' I can think of a few possibilities. One is to use a large language model to generate text that is based on external knowledge. This could be used to generate stories, articles, or other types of content. Another possibility is to use the large language model to generate questions and answers based on external knowledge. This could be used to create a knowledge base or to answer questions posed by users. Finally, the large language model could be used to generate natural language responses to user queries.'

In [56]:
# Check the summaries given to the chain
print(conversation.memory.buffer)


The human greeted the AI and the AI responded with a greeting and asked how it could help. The human then expressed interest in exploring the potential of integrating Large Language Models with external knowledge, to which the AI responded positively and asked for more information. The AI then suggested a few possibilities, such as using the large language model to generate text based on external knowledge, generate questions and answers based on external knowledge, or generate natural language responses to user queries.


## Conversation Buffer Window Memory

Acts in the same way as our earlier “buffer memory” but adds a window to the memory. Meaning that we only keep a given number of past interactions before “forgetting” them.

In [57]:
conversation = ConversationChain(llm = llm,
                                 memory = ConversationBufferWindowMemory(k = 1)) #The model remembers the k latest interactions

In [58]:
count_tokens(conversation,
             "Good morning AI!")

Spent a total of 85 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [59]:
count_tokens(conversation,
             "My interest here is to explore the potential of integrating Large Language Models with external knowledge")

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [60]:
count_tokens(conversation,
             "I just want to analyze the different possibilities. What can you think of?")

Spent a total of 233 tokens


' There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model better understand the context of a given text, or to help it generate more accurate results.'

In [61]:
count_tokens(conversation, 
             "What is my aim again?")

Spent a total of 169 tokens


' Your aim is to analyze the different possibilities for integrating Large Language Models with external knowledge.'

In [62]:
# See the buffer history
bufw_history = conversation.memory.load_memory_variables(inputs = [])['history']

In [63]:
print(bufw_history)

Human: What is my aim again?
AI:  Your aim is to analyze the different possibilities for integrating Large Language Models with external knowledge.


## Conversation Summary Buffer Memory

It's a combination of the summary buffer and the conversation buffer window. Summarizes the earliest interactions while maintaining the {max_token_limit} most recent tokens of the conversation.

In [64]:
conversation_sum_bufw = ConversationChain(llm = llm,
                                          memory = ConversationSummaryBufferMemory(llm = llm,
                                                                                   max_token_limit = 650))

# Chapter 5. Retrieval Augmentation (Preventing Hallucination)

In [ ]:
!pip install -qU \
    datasets==2.12.0 \
    apache_beam \
    mwparserfromhell

In [ ]:
import os
import tiktoken
import pinecone

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from datasets import load_dataset

In [67]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]

os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [68]:
# Download knowledge base dataset
data = load_dataset("wikipedia", "20220301.simple", split='train[:1000]')
data

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235M [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to D:/Users/GReyes15/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1000
})

In [69]:
data[6]

{'id': '13',
 'url': 'https://simple.wikipedia.org/wiki/Alan%20Turing',
 'title': 'Alan Turing',
 'text': 'Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dub

Make chunks of the dataset

In [70]:
tokenizer = tiktoken.get_encoding('p50k_base')

In [71]:
# Create function for getting number of tokens
def tiktoken_len(text):
    tokens = tokenizer.encode(text,
                              disallowed_special = [])
    
    return len(tokens)

In [72]:
tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")

28

In [73]:
# Use LangChain to split the text on chunks based on number of tokens
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400,
                                               chunk_overlap = 20,
                                               length_function = tiktoken_len,
                                               separators = ["\n\n", "\n", " ", ""])

In [74]:
# Split the text into chunks
chunks = text_splitter.split_text(data[6]['text'])[:3]
chunks

['Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathison Turing, latter son o

In [75]:
# Check lenght of chunks
tiktoken_len(chunks[0]), tiktoken_len(chunks[1]), tiktoken_len(chunks[2])

(397, 304, 370)

In [76]:
# Use LangChain embedding model using OpenAI
embed = OpenAIEmbeddings(document_model_name = 'text-embedding-ada-002',
                         query_model_name = 'text-embedding-ada-002',
                         openai_api_key = API_key)

D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! document_model_name is not default parameter.
                    document_model_name was transferred to model_kwargs.
                    Please confirm that document_model_name is what you intended.
  warnings.warn(
D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! query_model_name is not default parameter.
                    query_model_name was transferred to model_kwargs.
                    Please confirm that query_model_name is what you intended.
  warnings.warn(


In [77]:
# Embedd the text
texts = ['this is the first chunk of text',
         'then another second chunk of text is here']

res = embed.embed_documents(texts)

len(res), len(res[0])

(2, 1536)

In [ ]:
# Create embedding database on Pinecone (substitue to BQ or Cloud Storage)
index_name = 'langchain-retrieval-augmentation'

pinecone.init(api_key = "a66bd3e9-cb62-409b-8842-c2cdb5fc1a7d",
              environment = 'gcp-starter')

# Create new index
pinecone.create_index(name = index_name,
                      metric = 'dotproduct',
                      dimension = len(res[0]))

In [ ]:
index = pinecone.GRPCIndex(index_name)

index.describe_index_stats()

In [ ]:
# Fill the embedding space
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100
texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # Get metadata fields for each record
    metadata = {'wiki-id': str(record['id']),
                'source': record['url'],
                'title': record['title']}
    
    # Create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
    
    # Create individual metadata dicts for each chunk
    record_metadatas = [{'chunk': j,
                         'text': text,
                         **metadata} for j, text in enumerate(record_texts)]
    
    # Append the metadata to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    
    # Add texts if reaching the batch_limit
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts,
                                       chunk_size = 16)
        index.upsert(vectors = zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

In [ ]:
# Check the stats
index.describe_index_stats()

In [ ]:
# Connect the embedding space with LangChain
text_field = 'text'

index = pinecone.Index(index_name)

vectorstore = Pinecone(index,
                       embed.embed_query,
                       text_field)

In [ ]:
# Test with similarity search
query = 'Who was Benito Mussolini?'

vectorstore.similarity_search(query,
                              k = 3)

Chain_type documentation: https://python.langchain.com/docs/modules/chains/document/stuff

In [ ]:
# Completion LLM
llm = AzureChatOpenAI(deployment_name = 'gpt-35-turbo',
                      model_name = 'gpt-35-turbo',
                      temperature = 0)

qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = "stuff",
                                 retriever = vectorstore.as_retriever())

In [ ]:
qa.run(query)

In [ ]:
# Add citations to the answer
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(llm = llm,
                                                              chain_type = "stuff",
                                                              retriever = vectorstore.as_retriever())

In [ ]:
qa_with_sources(query)

In [ ]:
# Delete the Pinecone index
pinecone.delete_index(index_name)

# Chapter 6. AI Agents

LangChain agents are tools that help LLM models to work on tasks that are difficutl for them to run, like online web search, or mathematical operations.

In [79]:
import os

from langchain.llms import AzureOpenAI
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents import Tool, load_tools, initialize_agent, create_sql_agent
from langchain.prompts import PromptTemplate
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import Wikipedia, SerpAPIWrapper
from langchain.agents.react.base import DocstoreExplorer

In [80]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [81]:
# Load base LLM model
llm = AzureOpenAI(deployment_name = "text-davinci-003",
                  model_name = "text-davinci-003",
                  temperature = 0)

In [82]:
# Initialize a tool for the LLM model. In this case a calculator
llm_math = LLMMathChain(llm = llm)

# Initialize the math tool (deprecated, better to use it as a chain like in Chapter 3)
math_tool = Tool(name = 'Calculator',
                 func = llm_math.run,
                 description = 'Useful for when you need to answer question about math.')

# Pass all the tools to the LLM inside a list
tools = [math_tool]

D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [83]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer question about math.')

In [84]:
# You can also load prebuilt agents
tools = load_tools(['llm-math'],
                   llm = llm)

In [85]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [86]:
# Initialize the agent
zero_shot_agent = initialize_agent(agent = 'zero-shot-react-description',
                                   tools = tools,
                                   llm = llm,
                                   verbose = True,
                                   max_iterations = 3)

In [87]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [88]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0
Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0
Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

In [89]:
# We need a tool for every task we ask the agent
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 I need to look up the answer
Action: Look up
Action Input: Capital of Norway
Observation: Look up is not a valid tool, try one of [Calculator].
Thought: I need to look up the answer in a different way
Action: Calculator
Action Input: Capital of Norway

ValueError: unknown format from LLM: This question cannot be answered using the numexpr library, as it is not a mathematical expression.

In [90]:
prompt = PromptTemplate(input_variables = ['query'],
                        template = "{query}")

llm_chain = LLMChain(llm = llm,
                     prompt = prompt)

llm_tool = Tool(name = 'Language Model',
                func = llm_chain.run,
                description = 'Use this tool for general purpose queries and logic.')

# Add the new tool
tools.append(llm_tool)

# Reinitialize the agent
zero_shot_agent = initialize_agent(agent = 'zero-shot-react-description',
                                   tools = tools,
                                   llm = llm,
                                   verbose = True,
                                   max_iterations = 3)

In [91]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 I need to find the capital of Norway
Action: Language Model
Action Input: "What is the capital of Norway?"
Observation: 

The capital of Norway is Oslo.
Thought: I now know the final answer
Final Answer: The capital of Norway is Oslo.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'The capital of Norway is Oslo.'}

## Agent Types - Zero Shot ReAct

Basic agent to perform zero-shot tasks that do not require memory. This means that the agent considers the isngle interaction given by the user.

### Testing Database for SQL Agent
This example will use a SQL agent, so we need to create and connect a database for the agent

In [92]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [93]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table("stocks",
               metadata_obj,
               Column("obs_id", Integer, primary_key=True),
               Column("stock_ticker", String(4), nullable=False),
               Column("price", Float, nullable=False),
               Column("date", Date, nullable=False),)

In [94]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [95]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [96]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [97]:
for obs in observations:
    insert_obs(obs)

In [98]:
db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain_experimental\sql\base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


### Continue with Zero Shot ReAct

In [99]:
# Create the SQL tool
sql_tool = Tool(name = 'Stock DB',
                func = sql_chain.run,
                description = 'useful for when you need to answer questions about stocks and their prices')

In [100]:
tools = load_tools(['llm-math'],
                   llm = llm)

# Add SQL db tool
tools.append(sql_tool)

In [101]:
zero_shot_agent = initialize_agent(agent = 'zero-shot-react-description',
                                   tools = tools,
                                   llm = llm,
                                   verbose = True,
                                   max_iterations = 3)

In [102]:
result = zero_shot_agent("""What is the multiplication of the ratio between stock prices
for 'ABC' and 'XYZ' in January 3rd and the ratio between the
same stock prices in January the 4th?""")



> Entering new AgentExecutor chain...
 I need to compare the stock prices of ABC and XYZ on two different days
Action: Stock DB
Action Input: Stock prices of ABC and XYZ on January 3rd and January 4th

> Entering new SQLDatabaseChain chain...
Stock prices of ABC and XYZ on January 3rd and January 4th
SQLQuery:SELECT stock_ticker, price, date FROM stocks WHERE (stock_ticker = 'ABC' OR stock_ticker = 'XYZ') AND (date = '2023-01-03' OR date = '2023-01-04')
SQLResult: [('ABC', 232.0, '2023-01-03'), ('ABC', 225.0, '2023-01-04'), ('XYZ', 798.0, '2023-01-03'), ('XYZ', 795.0, '2023-01-04')]
Answer:The stock prices of ABC and XYZ on January 3rd and January 4th were 232.0 and 225.0 for ABC, and 798.0 and 795.0 for XYZ, respectively.
> Finished chain.

Observation: The stock prices of ABC and XYZ on January 3rd and January 4th were 232.0 and 225.0 for ABC, and 798.0 and 795.0 for XYZ, respectively.
Thought: I need to calculate the ratio between the two stock prices
Action: Calculator
Action Inp

In [103]:
# Look at the prompt of the agent
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Stock DB: useful for when you need to answer questions about stocks and their prices

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Stock DB]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## Conversational ReAct

Agent with memory. Useful for use cases that need to remember previous interations in a conversation or interaction.

In [104]:
# Initialize the memory buffer
memory = ConversationBufferMemory(memory_key = 'chat_history')

In [105]:
# Initialize the agent with the memory buffer
conversational_agent = initialize_agent(agent = 'conversational-react-description',
                                        tools = tools,
                                        llm = llm,
                                        verbose = True,
                                        max_iterations = 3,
                                        memory = memory)

In [106]:
result = conversational_agent("Provide me with the stock prices for ABC on January the 1st")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC on January the 1st

> Entering new SQLDatabaseChain chain...
ABC on January the 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01'
SQLResult: [(200.0,)]
Answer:The price of ABC on January the 1st was 200.0.
> Finished chain.

Observation: The price of ABC on January the 1st was 200.0.
Thought: Do I need to use a tool? No
AI: Is there anything else I can help you with?

> Finished chain.


In [107]:
# Ask again without specifying the date
result = conversational_agent("What are the stock prices for XYZ on the same day?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: Stock prices for XYZ on January 1st

> Entering new SQLDatabaseChain chain...
Stock prices for XYZ on January 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-01' LIMIT 5;
SQLResult: [(810.0,)]
Answer:The stock price for XYZ on January 1st was 810.0.
> Finished chain.

Observation: The stock price for XYZ on January 1st was 810.0.
Thought: Do I need to use a tool? No
AI: Is there anything else I can help you with?

> Finished chain.


In [108]:
# How does it uses the memory?
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [109]:
# This agent is built for conversational purposes, and uses more difficult approaches for multi-step procedures
result = conversational_agent("""What is the multiplication of the ratio between stock prices
for 'ABC' and 'XYZ' in January 3rd and the ratio between the
same stock prices in January the 4th?""")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: Get the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th

> Entering new SQLDatabaseChain chain...
Get the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th
SQLQuery:SELECT (SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-03') / (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-03') AS ratio_jan_3, (SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-04') / (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-04') AS ratio_jan_4 FROM stocks LIMIT 1;
SQLResult: [(0.2907268170426065, 0.2830188679245283)]
Answer:The ratio between stock prices for 'ABC' and 'XYZ' in January 3rd is 0.2907268170426065 and the ratio between the same stock prices in Ja

## ReAct Docstore

Agent built for information search and lookup. It allows to store and retrieve information using different retrieval methods, like Wikipedia.

In [110]:
docstore = DocstoreExplorer(Wikipedia())

tools = [Tool(name = 'Search',
              func = docstore.search,
              description = 'Search Wikipedia'),
         Tool(name = 'Lookup',
              func = docstore.lookup,
              description = 'Lookup a term in Wikipedia')]

In [111]:
# Initialize the agent
docstore_agent = initialize_agent(agent = 'react-docstore',
                                  tools = tools,
                                  llm = llm,
                                  verbose = True,
                                  max_iterations = 3)

In [112]:
docstore_agent("What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (, ARK-ihm-EE-deez; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.

{'input': "What were Archimedes' last words?",
 'output': 'Do not disturb my circles'}

## Self-Ask with Search

Agent that performs searches and asks follow-up questions as often as required to get a final answer.

This agent requires a SerpAPI key. Currently using **personal free API key** for testing purposes. Limited to **100 searches/month**

In [113]:
# Initialize the search chain
path = "SerpAPI_key.txt"
with open(path) as f:
    SerpAPI_key = f.readlines()[0]

search = SerpAPIWrapper(serpapi_api_key = SerpAPI_key)

In [114]:
# Create the search tool
tools = [Tool(name = 'Intermediate Answer',
              func = search.run,
              description = 'Google search')]

In [115]:
# Initialize the agent
self_ask_with_search = initialize_agent(agent = 'self-ask-with-search',
                                  tools = tools,
                                  llm = llm,
                                  verbose = True)

In [116]:
self_ask_with_search('Who lived longer: Plato, Socrates, or Aristotle?')



> Entering new AgentExecutor chain...
 Yes.
Follow up: How old was Plato when he died?
Intermediate answer: eighty
Follow up: How old was Socrates when he died?
Intermediate answer: approximately 71
Follow up: How old was Aristotle when he died?
Intermediate answer: 62 years
So the final answer is: Plato

> Finished chain.


{'input': 'Who lived longer: Plato, Socrates, or Aristotle?',
 'output': 'Plato'}

# Chapter 7. Build Custom Tools for LLM Agents

In [117]:
import os
import torch
import requests

from langchain.tools import BaseTool
from math import pi, sqrt, cos, sin
from typing import Union
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent
from typing import Optional
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

In [118]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

## Tools with Single Parameters

In [119]:
# Create a tool that calculates a circle circumference
class CircumferenceTool(BaseTool):
    name = 'Circumference calculator'
    description = "Use this tool when you need to calculate a circumference using the radius of a circle"
    
    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi
    
    # Asynchronous run (not covered in the chapter)
    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [120]:
# Initialize the LLM model
llm = AzureChatOpenAI(deployment_name = 'gpt-35-turbo',
                      model_name = 'gpt-35-turbo',
                      temperature = 0)

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(memory_key = 'chat_history',
                                                       k = 5,
                                                       return_messages = True)

# Initizalize the agent
tools = [CircumferenceTool()]

agent = initialize_agent(agent = 'chat-conversational-react-description',
                         tools = tools,
                         llm = llm,
                         verbose = True,
                         max_iterations = 3,
                         early_stopping_method = 'generate',
                         memory = conversational_memory)

In [121]:
# Current agent ignores the Circumference tool
agent("Calculate the circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The circumference of the circle is approximately 49.03mm."
}

> Finished chain.


{'input': 'Calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of the circle is approximately 49.03mm.'}

In [122]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [123]:
# Modify the prompt to determine the agent to use tools for doing math
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist."""

In [124]:
new_prompt = agent.agent.create_prompt(system_message = sys_msg,
                                       tools = tools)

agent.agent.llm_chain.prompt = new_prompt

In [125]:
agent("Calculate the circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}
```

> Finished chain.


{'input': 'Calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [HumanMessage(content='Calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of the circle is approximately 49.03mm.')],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}

## Tools with Multiple Parameters

In [126]:
# Build a hypotenuse calculator given a combination of triangle side lenghts and/or angles
desc = ("""Use this tool when you need to calculate the length of a hypotenuse
given one or two sides of a triangle and/or an angle (in degrees).
To use the tool, you must provide at least two of the following parameters:
['adjacent_side', 'opposite_side', 'angle'].""")

class PythagorasTool(BaseTool):
    name = 'Hypotenuse calculator'
    description = desc
    
    def _run(self,
             adjacent_side: Optional[Union[int, float]] = None,
             opposite_side: Optional[Union[int, float]] = None,
             angle: Optional[Union[int, float]] = None):
        
        # Check for the given values
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
        
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
        
tools = [PythagorasTool()]

In [127]:
# Update the agent's prompt
new_prompt = agent.agent.create_prompt(system_message = sys_msg,
                                       tools = tools)

agent.agent.llm_chain.prompt = new_prompt

# Update also the tools
agent.tools = tools

In [128]:
agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...
{
    "action": "Hypotenuse calculator",
    "action_input": {
        "adjacent_side": 51,
        "opposite_side": 34
    }
}
Observation: 61.29437168288782
Thought:{
    "action": "Final Answer",
    "action_input": "The length of the hypotenuse is approximately 61.29cm."
}

> Finished chain.


{'input': 'If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?',
 'chat_history': [HumanMessage(content='Calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of the circle is approximately 49.03mm.'),
  HumanMessage(content='Calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.')],
 'output': 'The length of the hypotenuse is approximately 61.29cm.'}

## Create captions for online images

Create a new tool based on HuggingFace code for image captioning

In [129]:
# Specify model
hf_model = "Salesforce/blip-image-captioning-large"

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Preprocessor will prepare images for the model
processor = BlipProcessor.from_pretrained(hf_model)

# Initialize the LLM model
model = BlipForConditionalGeneration.from_pretrained(hf_model).to(device)

In [130]:
desc = ("""Use this tool when given the URL of an image that you'd like to be
described. It will retrun a simple caption describing the image.""")

class ImageCaptionTool(BaseTool):
    name = 'Image captioner'
    description = desc
    
    def _run(self, url: str):
        # Download the image and convert to PIL object
        image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
        
        # Preprocess the image
        inputs = processor(image, return_tensors = 'pt').to(device)
        
        # Generate the caption
        out = model.generate(**inputs, max_new_tokens=20)
        
        # Get the caption
        caption = processor.decode(out[0], skip_special_tokens=True)
        
        return caption
    
    def _arun(self, query: str):
        raise notImplementedError("This tool does not support async")
        
tools = [ImageCaptionTool()]

In [131]:
# Reinitialize the agent prompt
sys_msg = """Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a poerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist."""

new_prompt = agent.agent.create_prompt(system_message = sys_msg,
                                       tools = tools)
agent.agent.llm_chain.prompt = new_prompt

# Update the agent's tools
agent.tools = tools

In [132]:
# Test the tool
img_url = "https://image.adsoftheworld.com/56agha7b3d768gu5e37rl4ji08pz"
agent(f"What does this image show?\n{img_url}")



> Entering new AgentExecutor chain...
{
    "action": "Image captioner",
    "action_input": "https://image.adsoftheworld.com/56agha7b3d768gu5e37rl4ji08pz"
}
Observation: there is a figurine of a dog on a plate with a knife and fork
Thought:{
    "action": "Final Answer",
    "action_input": "The image shows a figurine of a dog on a plate with a knife and fork."
}

> Finished chain.


{'input': 'What does this image show?\nhttps://image.adsoftheworld.com/56agha7b3d768gu5e37rl4ji08pz',
 'chat_history': [HumanMessage(content='Calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of the circle is approximately 49.03mm.'),
  HumanMessage(content='Calculate the circumference of a circle that has a radius of 7.81mm'),
  AIMessage(content='The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'),
  HumanMessage(content='If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?'),
  AIMessage(content='The length of the hypotenuse is approximately 61.29cm.')],
 'output': 'The image shows a figurine of a dog on a plate with a knife and fork.'}

# Chapter 8. Agents with Long-Term-Memory

In [133]:
import os
import pinecone

from datasets import load_dataset
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.agents import Tool
from langchain.agents import initialize_agent

## Build the knowledge base

In [134]:
data = load_dataset('squad', split='train')
data

Found cached dataset squad (D:/Users/GReyes15/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [135]:
# Remove duplicates
data = data.to_pandas()
data.drop_duplicates(subset='context', keep='first', inplace=True)
data.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."


### Initialize Embedding Model and vector DB

In [136]:
# Initialize Azure OpenAI
path = "API_key.txt"
with open(path) as f:
    API_key = f.readlines()[0]
    
os.environ['OPENAI_API_KEY'] = API_key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'azure-openai-api-url'

In [137]:
# Use LangChain embedding model using OpenAI
embed = OpenAIEmbeddings(document_model_name = 'text-embedding-ada-002',
                         query_model_name = 'text-embedding-ada-002',
                         openai_api_key = API_key)

D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! document_model_name is not default parameter.
                    document_model_name was transferred to model_kwargs.
                    Please confirm that document_model_name is what you intended.
  warnings.warn(
D:\Users\GReyes15\Anaconda3\lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! query_model_name is not default parameter.
                    query_model_name was transferred to model_kwargs.
                    Please confirm that query_model_name is what you intended.
  warnings.warn(


In [ ]:
# Create embedding database on Pinecone (substitue to BQ or Cloud Storage)
index_name = 'langchain-retrieval-agent'

pinecone.init(api_key = "pinecone-api-key",
              environment = 'gcp-starter')

if index_name not in pinecone.list_indexes():
    # Create the new index
    pinecone.create_index(name = index_name,
                          metric = 'dotproduct',
                          dimension = 1536)

In [ ]:
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

## Indexing

In [ ]:
# Fill the embedding space
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100
texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # Get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    
    # Get metadata for this record
    metadatas = [{'title': record['title'],
                  'text': record['context']} for j, record in batch.iterrows()]
    
    # Get the list of contexts/documents
    documents = batch['context']
    
    # Create embeddings
    embeds = embed.embed_documents(documents,
                                   chunk_size = 16)
    
    # Get IDS
    ids = batch['id']
    
    # Add to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

In [ ]:
index.describe_index_stats()

## Creating a Vector Store and Querying

In [ ]:
text_field = 'text'

# Switch back to normal index for Langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(index,
                       embed.embed_query,
                       text_field)

In [ ]:
# Use similarity search for testing semantic search
query = 'when was the college of engineering in the University of Notre Dame established?'

vectorstore.similarity_search(query,
                              k = 3)

## Initialize Conversational Agent

In [ ]:
# Initialize the LLM model
llm = AzureChatOpenAI(deployment_name = 'gpt-35-turbo',
                      model_name = 'gpt-35-turbo',
                      temperature = 0)

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(memory_key = 'chat_history',
                                                       k = 5,
                                                       return_messages = True)

# Retreival QA chain
qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = "stuff",
                                 retriever = vectorstore.as_retriever())

In [ ]:
# Test the previous query
qa.run(query)

In [ ]:
# Convert the chain into a Tool
tools = [Tool(name = 'Knowledge Base',
              func = qa.run,
              description = "Use this tool when answering general knowledge queries to get more information about the topic.")]

In [ ]:
# Initialize the agent
agent = initialize_agent(agent = 'chat-conversational-react-description',
                         tools = tools,
                         llm = llm,
                         verbose = True,
                         max_iterations = 3,
                         early_stopping_method = 'generate',
                         memory = conversational_memory)

### Using the Conversational Agent

In [ ]:
agent(query)

In [ ]:
agent("what is 2 * 7?")

In [ ]:
agent("can you tell me some facts about the University of Notre Dame?")

In [ ]:
agent("can you summarize these facts in two short sentences")

In [ ]:
# Delete the Pinecone index
pinecone.delete_index(index_name)

# Chapter 9. Langchain for GCP Vertex AI

In [139]:
import requests

from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, load_tools
from transformers import BlipProcessor, BlipForConditionalGeneration

from google.cloud import aiplatform
from google.oauth2 import service_account
from PIL import Image

In [140]:
# Initialize connection to GCP VertexAI
credentials = service_account.Credentials.from_service_account_file('gcp-labx-aba-461f9042302b.json')

aiplatform.init(project = 'gcp-labx-aba',
                credentials = credentials)

In [ ]:
# Initialize the LLM model
llm = VertexAI(model_name = 'text-bison@001',
               temperature = 0)

# Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(memory_key = 'chat_history',
                                                       k = 5,
                                                       return_messages = True)

In [ ]:
# Initizalize the tools (math and image caption)
tools = load_tools(['llm-math'],
                   llm = llm)

hf_model = "Salesforce/blip-image-captioning-large"
device = 'cpu'

# preprocessor will prepare images for the model
processor = BlipProcessor.from_pretrained(hf_model)
# then we initialize the model itself
model = BlipForConditionalGeneration.from_pretrained(hf_model).to(device)

desc = ("""Use this tool when given the URL of an image that you'd like to be described.
It will return a simple caption describing the image.""")

class ImageCaptionTool(BaseTool):
    name = "Image captioner"
    description = desc
    
    def _run(self, url: str):
        # download the image and convert to PIL object
        image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
        # preprocess the image
        inputs = processor(image, return_tensors="pt").to(device)
        # generate the caption
        out = model.generate(**inputs, max_new_tokens=20)
        # get the caption
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools.append(ImageCaptionTool())

In [ ]:
# Initialize the agent
agent = initialize_agent(agent = 'chat-conversational-react-description',
                         tools = tools,
                         llm = llm,
                         verbose = True,
                         max_iterations = 3,
                         early_stopping_method = 'generate',
                         memory = conversational_memory)

# Make sure the agent uses the math tool
sys_msg = """Assistant is a large language model trained by VertexAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist."""

new_prompt = agent.agent.create_prompt(system_message = sys_msg,
                                       tools=tools)

agent.agent.llm_chain.prompt = new_prompt

In [ ]:
agent("How much is 8^16?")

In [ ]:
img_url = "https://image.adsoftheworld.com/56agha7b3d768gu5e37rl4ji08pz"
agent(f"What does this image show?\n{img_url}")